In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-09 01:31:53--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  44.6MB/s    in 48s     

2021-01-09 01:32:41 (38.8 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-09 01:32:41--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [ ]:
!cp ../drive/MyDrive/slt/early-fusion-v3/*.py signjoey/

In [ ]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [ ]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:19<00:00, 89.10it/s]


In [ ]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]
list_doc['training']['recognition_loss_weight'] = 20

list_doc["model"]["embeddings"] = []
for i in range(2):
  list_doc["model"]["embeddings"].append(list_doc["model"]["encoder"]["embeddings"].copy())

list_doc["model"]["encoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["encoder"]["hidden_size"] = 1024
list_doc["model"]["decoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["decoder"]["hidden_size"] = 1024

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
%cd slt

/content/slt


In [ ]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
	BLEU-4 15.76	(BLEU-1: 40.67,	BLEU-2: 27.01,	BLEU-3: 19.99,	BLEU-4: 15.76)
	CHRF 38.95	ROUGE 41.00
2021-01-09 02:39:22,177 Logging Recognition and Translation Outputs
2021-01-09 02:39:22,177 ========================================================================================================================
2021-01-09 02:39:22,177 Logging Sequence: dev/26August_2009_Wednesday_tagesschau-3225
2021-01-09 02:39:22,178 	Gloss Reference :	FREITAG SUEDOST SONNE FREITAG DANN MEHR WOLKE REGEN KOENNEN
2021-01-09 02:39:22,178 	Gloss Hypothesis:	FREITAG SUEDOST ***** FREITAG **** **** ***** ***** KOENNEN
2021-01-09 02:39:22,178 	Gloss Alignment :	                D             D    D    D     D            
2021-01-09 02:39:22,178 	--------------------------------------------------------------------------------------------------------------------
2021-01-09 02:39:22,183 	Text Reference  :	am freitag *** in der südosthälfte zum teil  noch *******

In [ ]:
33.87 & 19.61 & 34.28 & 19.42